<a href="https://colab.research.google.com/github/Miksan31/Agentic-AI/blob/main/Medical_Research_Agent_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Install project dependencies
!pip install crewai_tools langchain_openai langchain_groq langchain_anthropic langchain_community cohere --quiet


In [ ]:
# @title LLM API Key

import os
from getpass import getpass
from crewai import Agent, Task, Crew, Process
from textwrap import dedent
from langchain_openai import ChatOpenAI

# os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAPI key: ")
# ↑ uncomment to use OpenAI's API
os.environ["GROQ_API_KEY"] = getpass("Enter GROQ_API_KEY: ")
# ↑ uncomment to use Groq's API

# Check if the 'output-files' directory exists, and create it if it doesn't
if not os.path.exists('output-files'):
    os.makedirs('output-files')

In [ ]:
# @title Serper API Key

from crewai_tools import SerperDevTool

os.environ["SERPER_API_KEY"] = getpass("Serper Key")

search_tool = SerperDevTool()

In [ ]:
# @title Agents

from crewai import LLM
# Agent Definitions

senior_research_analyst= Agent(
    role= "Senior Research Analyst",
    goal= "Research and analyze the medical industry using Generative AI on {topic} from reliable web sources",
    backstory= "A highly skilled and experienced Senior Research Analyst with a deep understanding of Generative AI and its applications in the medical industry. They have a proven track record of conducting thorough research and analysis, and are able to identify trends and patterns in complex data sets. They are also proficient in using various tools and technologies to support their research, including Generative AI and other advanced analytics techniques.",
    allow_delegation= False,
    verbose= True,
    # ↑ Whether the agent execution should be in verbose mode
    max_iter=3,
    # ↑ maximum number of iterations the agent can perform before being forced to give its best answer (generate the output)
    max_rpm=100, # This is the maximum number of requests per minute that the agent can make to the language model
    # llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.8)

    llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    # model="groq/llama-3.3-70b-specdec",
    temperature=0.7
    )
)

content_writer= Agent(
    role= "Content Writer",
    goal= "Write a comprehensive report on the medical industry using Generative AI on {topic} based on the research conducted by the Senior Research Analyst",
    backstory= "A highly skilled and experienced Content Writer with a deep understanding of the medical industry and its applications in Generative AI. They have a proven track record of writing clear, concise, and engaging reports that effectively communicate complex information to a wide range of audiences. They are also proficient in using various tools and technologies to support their writing, including Generative AI and other advanced analytics techniques.",
    allow_delegation= False,
    verbose= True,
    # ↑ Whether the agent execution should be in verbose mode
    max_iter=3,
    # ↑ maximum number of iterations the agent can perform before being forced to give its best answer (generate the output)
    max_rpm=100, # This is the maximum number of requests per minute that the agent can make to the language model
    # llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.8)
    llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    # model="groq/llama-3.3-70b-specdec",
    temperature=0.7
    )
)


In [ ]:
# @title Tasks
# Task Definitions

import datetime

research_task= Task(
    description= "Conduct research on the medical industry using Generative AI on {topic}, evaluate source credibitlity and fact check all information, include all relevant citations and sources",
    expected_output= "A comprehensive report on the medical industry using Generative AI on {topic}, including all relevant citations and sources,clear catergorization of main themes and patterns, with proper formatting",
    agent=senior_research_analyst,
    output_file=f'output-files/agent_1_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.md'
    # ↑ The output of each task iteration will be saved here
)

writing_task= Task(
    description= "Write a report on the medical industry using Generative AI on {topic}, include all relevant citations and sources, clear catergorization of main themes and patterns, with proper formatting",
    expected_output= "A comprehensive blog post on the medical industry using Generative AI on {topic}, including all relevant citations and sources,clear catergorization of main themes and patterns, with proper formatting",
    agent=content_writer,
    context=[research_task],
    # ↑ specify which task's output should be used as context for subsequent tasks
    output_file=f'output-files/agent_2_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.md'
    # ↑ The output of each task iteration will be saved here
)


In [ ]:
# @title Define variables and input them
print("Agentic AI Crew")
print('-------------------------------------------')
var_1 = input("Research Topic\n")

In [ ]:
# @title  Crew build
def main():
    # Instantiating crew with a sequential process
    crew = Crew(
        agents=[senior_research_analyst, content_writer],
        tasks=[research_task, writing_task],
        verbose=True,
        # ↑ indicates the verbosity level for logging during execution.
        process=Process.sequential
        # ↑ the process flow that the crew will follow (e.g., sequential, hierarchical).
    )

    inputs = (var_1)

    result = crew.kickoff(inputs= {"topic" : inputs})
    print("\n\n########################")
    print("## Custom crew run result:")
    print("########################\n")
    print(result)

    return result

if __name__ == "__main__":
  result = main()

In [ ]:
# @title Results of crew as markdown
from IPython.display import display, Markdown

markdown_text = result.raw  # Adjust this based on the actual attribute

# Display the markdown content
display(Markdown(markdown_text))